In [2]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [407 kB]
Get:12 https://c

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33605939| RGYFDX8QXKEIR|B007KO2MLO|     328837464|           Big Maria|Digital_Ebook_Pur...|          4|            0|          0|   N|                N|              Quirky|Elmore Leonard me...| 2013-09-09|
|         US|   34058393|R13CBGTMNV9R8Z|B005FLODDE|     764276359|The Woman Who Was...|Digital_Ebook_Pur

In [5]:
#filter the dataset for total_votes being greater than or equal to 20

vote_count = df.filter("total_votes >= 20")
vote_count.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    4514126|R39ESX43X1SA5T|B006NG0GXQ|     402593554|Forever His: A Ti...|Digital_Ebook_Pur...|          1|           36|         47|   N|                Y|   Completely absurd|From the time tha...| 2013-09-09|
|         US|   45292583|R31QRZ3JAMPBB4|B0078T6YHY|     267532424|A Lady in Defianc...|Digital_Ebook_Pur

In [6]:
#create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
new_table = vote_count.filter(vote_count["helpful_votes"]/ vote_count["total_votes"]>= 0.5)
new_table.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    4514126|R39ESX43X1SA5T|B006NG0GXQ|     402593554|Forever His: A Ti...|Digital_Ebook_Pur...|          1|           36|         47|   N|                Y|   Completely absurd|From the time tha...| 2013-09-09|
|         US|   45292583|R31QRZ3JAMPBB4|B0078T6YHY|     267532424|A Lady in Defianc...|Digital_Ebook_Pur

In [7]:
# Create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
new_table.filter(new_table["vine"] == "Y").show()
new_table.filter(new_table["vine"] == "Y").count()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



0

In [8]:
#Create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'N'
new_table.filter(new_table["vine"] == "N").show(5)
new_table.filter(new_table["vine"] == "N").count()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    4514126|R39ESX43X1SA5T|B006NG0GXQ|     402593554|Forever His: A Ti...|Digital_Ebook_Pur...|          1|           36|         47|   N|                Y|   Completely absurd|From the time tha...| 2013-09-09|
|         US|   45292583|R31QRZ3JAMPBB4|B0078T6YHY|     267532424|A Lady in Defianc...|Digital_Ebook_Pur

65149

In [9]:
# number of five star reviews
five_star = new_table.filter(new_table["star_rating"]== 5)
five_star.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12175486|R23FKBEXURC3SN|B00F225YK2|     953280558|Very Bad Things (...|Digital_Ebook_Pur...|          5|           28|         36|   N|                N|Holy Hotness, Bla...|Another Fave of 2...| 2013-09-09|
|         US|   28507300| RTRVYV0GWUTCK|B00EY3DN58|     694178149|there was no Jesu...|Digital_Ebook_Pur

In [10]:
five_star.count()

24673

In [11]:
# total reviews
new_table.count()

65149

In [12]:
#percentage of five star reviews
five_star.count() / new_table.count()

0.37871648068274266

In [13]:
#percentage of paid 5 star reviews
# The result will be an error because there are no paid reviews in this dataset being analyzed. Hence there were no paid vine reviews in this dataset 
five_star.filter(five_star["vine"] == "Y").count()/new_table.filter(new_table["vine"] == "Y").count()

ZeroDivisionError: ignored

In [14]:
# percentage of unpaid 5 star reviews 
five_star.filter(five_star["vine"] == "N").count()/new_table.filter(new_table["vine"] == "N").count()

0.37871648068274266